# Decision Tree Regressor

Let's first import necessary libraries.
Pandas is used for data frame handling.
Numpy is for used for numercial analysis.
Sci-kit learn module provides various submodules to preprocess the data, create and train the machine learning model and to test it's performance.
matplotlib is used for plotting of graphs.
Tensorflow module provides various submodules to preprocess the data, create and train the machine learning model and to test it's performance.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer

Fisrt we import data from the csv file and save it as a pandas data frame in variable grd.


In [2]:
grd = pd.read_csv("../data/graphene_data_final.csv")

We take the 1st four columns Graphene_percentage, feed, RPM and DOC as input and MRR as 2st output and `Ra` as 2nd output.

In [3]:
X, Y = grd[['Graphene_percentage', 'FEED', 'RPM', 'DOC']], grd['MRR_gm_per_sec']
Y2 = grd['Ra']

Now we will split the input and output data to respective test and train data sets.
X_train is the input and Y_train is the output data set used to train the model to predict `MRR`.
X_test and Y_test are input and output data sets respectively those are used to test the performance of the `MRR` predictor model.
X_train2 is the input and Y2_train is the output data set used to train the model to predict `Ra`.
X_test2 and Y2_test are input and output data sets respectively those are used to test the performance of the `Ra` predictor model.

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=33)
X_train2, X_test2, Y2_train, Y2_test = train_test_split(X, Y2, test_size = 0.3, random_state=1)

`tree.DecisionTreeRegressor()` function is used to initiate a decision tree model object.
`fit` function is used to train the model using the train data sets given within them and them `score` function is used to test the performance of the model.

In [5]:
print(tree.DecisionTreeRegressor().fit(X_train,Y_train).score(X_test,Y_test))

0.8594779567262018


The above model is produced without any parameter tunning.
But we can improve the model by training and testing the models with various sets of multiple parameters and can get the best performing model to predict our output.
That's why now we create parameter grid that contains various possible values for parameters to find the best set of parameters.

In [6]:
param_grid = { 
    'min_samples_split': [2,4,8,12,16],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_leaf': [2,4,6,8],
    'max_depth' : [1,3,5,7,9,11,13,15],
}

Grid search algorithm streamlines the process of finding the best sets of parameters for a given model and provided when parameter grid and a basic model is given to it.

In [7]:
dtr_cv = tree.DecisionTreeRegressor(random_state=16)

GridsearchCV function creates a gridsearch object that takes another model object, parameter grid, number of cross validations to be done as input.
Upon fitting the gridsearch model object with train data, it trains and validates itself to find the best set of parameters from the given paramter grid. It is worth mentioning that grid search cv used cross validation algorithm to split the train data into a particular number of parts and uses one part to validate the model and other parts to train the model. At last it is able to find the highest scoring model and its parameters.

In [8]:
CV_dtr = GridSearchCV(estimator=dtr_cv, param_grid=param_grid, cv= 5)
CV_dtr.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=16),
             param_grid={'max_depth': [1, 3, 5, 7, 9, 11, 13, 15],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 8, 12, 16]})

As we can see grid search model acts a model itself since it inherits the basic model.

In [9]:
CV_dtr.score(X_test, Y_test)

0.8730574949203294

Using gridsearch we are able to find a model that performs 2% better than the original basic model. The improvement may seem miniscule because the train data set is small and the basic model itself applies certain algorithms to find the best parameters.

In [10]:
CV_dtr.best_params_

{'max_depth': 7,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 8}

This model is used to predict surface roughness(Ra).

In [11]:
dtr_cv2 = tree.DecisionTreeRegressor(random_state=6)

In [12]:
CV_dtr2 = GridSearchCV(estimator=dtr_cv2, param_grid=param_grid, cv= 5)
CV_dtr2.fit(X_train2, Y2_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=6),
             param_grid={'max_depth': [1, 3, 5, 7, 9, 11, 13, 15],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 8, 12, 16]})

In [13]:
CV_dtr2.score(X_test2, Y2_test)

0.34724831489740293

As we can see the test score is pretty low, we, therefore, can conclude that the gridsearch algorithm using the decision tree model is not able to find the actual relation between the input variables and surface roughness.

Now let's save the best performing decision tree models as a "Pickle" file.

In [14]:
import pickle
with open('../trained_models/decision_tree_.pkl','wb') as f:
    pickle.dump(CV_dtr,f)
with open('../trained_models/decision_tree_RA.pkl','wb') as f:
    pickle.dump(CV_dtr2,f)